In [2]:
import glob, os
import numpy as np
import pandas as pd
import soundfile as sf
from functools import partial
from pathlib import Path



import yamnet_model.params as yamnet_params
import yamnet_model.yamnet as yamnet_model

from src.features import FeatureExtractor


from omegaconf import OmegaConf

from src.data import load_metadata, find_paths
# from src.features.YAMNetExtractor import YAMNetExtractor
import src.utils

from omegaconf import OmegaConf
conf = OmegaConf.load("./config.yaml")

# Prepare topic task

We need to create audio fragments and yamnet embeddings for the topic task


## Step 1: Create audio files

In [3]:
# TODO DEZE code checken
from tts import tts

# Directory where audio files will be stored
query_audio_dir = os.path.join("../", conf.query_audio_dir)
target_dir = os.path.join("../", conf.dataset_path, 'topic_task')

# Create audio for train set
topics_df_path = os.path.join(target_dir, 'podcasts_2020_topics_train.xml')
create_audiofiles(topics_df_path, target_dir)

# Create audio for test set
topics_df_path = os.path.join(target_dir, 'podcasts_2020_topics_test.xml')
create_audiofiles(topics_df_path, target_dir)

# HIerna omzetten naar .ogg

## Step 2: Create yamnet embeddings

In [3]:

class YAMNetExtractor(FeatureExtractor):
    """Class for feature extraction with YAMNet.

    example:
    ex = YAMNetExtractor()
    ex.embedding(input_paths, output_paths, embed_paths)
    """

    def __init__(self, logfile="./log_yamnet", yamnet_path="./yamnet_class_map.csv"):
        """Init method for YAMNetExtractor."""
        super().__init__(logfile=logfile)
        self.model_checkpoint = os.path.join("./yamnet_model/yamnet.h5")
        
        #oud
        #self.class_names = os.path.join(
        #    os.getenv("YAMNET_PATH"), "yamnet_class_map.csv"
        #)
        
        # new
        print("init with yamnet path: ", yamnet_path)
        self.class_names = yamnet_path
        self.sample_rate = 44100

    def embedding(self, input_paths, output_paths):
        """Extract YAMnet features with opensmile using a single process."""
        save_embedding = True

        paths = list(zip(input_paths, output_paths))

        params = yamnet_params.Params(
            sample_rate=self.sample_rate, patch_hop_seconds=0.48
        )

        class_names = yamnet_model.class_names(self.class_names)
        yamnet = yamnet_model.yamnet_frames_model(params)
        yamnet.load_weights(self.model_checkpoint)

        func = partial(
            self._embed,
            yamnet=yamnet,
            params=params,
            class_names=class_names,
            save_embedding=save_embedding,
        )

        self.single_process(func, paths)

    @staticmethod
    def _embed(paths, yamnet, params, class_names, save_embedding=False):
        """Individual YAMnet extraction process."""

        input_path, embed_path = paths
        #print("[del] paths, should exist: ", input_path)
        input_path_exists, output_path_exists = FeatureExtractor.feature_path_checker(
            input_path, embed_path
        )
        
        print("[del] will it be true: ", input_path_exists and not output_path_exists)
        print('Should exist: ', input_path, input_path_exists)
        print('Should not exist: ', embed_path, output_path_exists)
        if input_path_exists and not output_path_exists:
            print("[del] if statement TRUE! reading ", input_path)
            wav_data, sr = sf.read(input_path, dtype=np.int16)
            print("[del] wav_data: ", wav_data)
            waveform = np.mean(wav_data, axis=1) / 32768.0

            approx_size = int(
                len(waveform) / params.sample_rate / params.patch_hop_seconds
            )  # approximate (overestimated) size of output
            embedding = np.zeros((approx_size, 1024))
            score = np.zeros((approx_size, 521))

            waveform_size = len(waveform)
            i = 0
            n_seconds = 300
            di = int(n_seconds * params.sample_rate)  # 5min segments

            real_size = 0
            while i <= waveform_size:
                scores, embeddings, spectrogram = yamnet(
                    waveform[i : i + di + int(0.47 * params.sample_rate)]
                ) # the 0.47s has to do with yamnet windowing
                scores = scores.numpy()
                print("[del] scores: ", scores)
                embeddings = embeddings.numpy()

                embedding[real_size : real_size + len(scores)] = embeddings
                score[real_size : real_size + len(scores)] = scores

                real_size += len(scores)
                i += di

            if save_embedding:
                print("[del] Will also save the embedding, to: ", embed_path)
                _, _ = FeatureExtractor.feature_path_checker(
                    input_path, embed_path
                )  # also create embed path if necessary

                df = pd.DataFrame(embedding)
                df["time (s)"] = np.arange(len(embedding)) * 0.48
                df.set_index("time (s)", inplace=True)
                df.astype(np.float16).to_hdf(
                    embed_path, "embedding", mode="w", complevel=6
                )
                del df

#             df = pd.DataFrame(score, columns=class_names)
#             df["time (s)"] = np.arange(len(score)) * 0.48
#             df.set_index("time (s)", inplace=True)
#             df.astype(np.float16).to_hdf(output_path, "score", mode="w", complevel=6)

            del df
            del embedding
            del score
            del spectrogram



In [4]:


def find_files(directory, ext='.ogg'):
    file_list = []
    for path in Path(directory).rglob('*{}'.format(ext)):

        file_list.append(str(path))
        
    return file_list


def get_output_paths(file_list, output_folder, ext=".h5"):
    
    
    outputs = []
    for f in file_list:
        spl = os.path.normpath(f).split(os.path.sep)
        new_lastname = spl[-1].split('.')[0] + ext

        filepath = os.path.join(output_folder, spl[-3], spl[-2], new_lastname)
        outputs.append(filepath)
    return outputs
        

In [5]:
# Load the configuration
# conf = OmegaConf.load("./config.yaml")
query_audio_dir = conf.query_audio_dir
# query_audio_dir = 'test'
print(query_audio_dir)

# Where the yamnet embeddings will be stored
output_folder = conf.query_embed_dir
print(output_folder)

# Find paths that need to be converted
# input_paths = find_paths(subset, input_path, ".ogg")
input_paths = find_files(query_audio_dir, ext='.ogg')
# input_paths = input_paths[-2:]
print(input_paths)
embed_paths = get_output_paths(input_paths, output_folder, ext=".h5")
# embed_paths = embed_paths[-2:]
print(embed_paths)

../data/sp/podcasts-no-audio-13GB/topic_task/query_audio
../data/sp/yamnet/query_embedding
['../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/12.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/13.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/39.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/11.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/10.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/38.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/14.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/28.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/29.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/15.ogg', '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/descriptio

In [6]:
THE_YAMNET_PATH = './yamnet_model/yamnet_class_map.csv'

# Run the YAMnet feature extraction
ex = YAMNetExtractor(logfile=os.path.join(conf.features_output_path, "log_yamnet"), yamnet_path = THE_YAMNET_PATH)
ex.embedding(input_paths, embed_paths)

init with yamnet path:  ./yamnet_model/yamnet_class_map.csv


  0%|          | 0/124 [00:00<?, ?it/s]

[del] will it be true:  True
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/12.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/description/fast/12.h5 False
[del] if statement TRUE! reading  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/12.ogg
[del] wav_data:  [[0 0]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/io/pytables.py:314: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->axis0] [items->None]

  f(store)
/opt/miniconda3/envs/troep/lib/python3.8/site-packages/pandas/io/pytables.py:314: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->integer,key->block0_items] [items->None]

  f(store)
  1%|          | 1/124 [00:00<00:44,  2.76it/s]

[del] scores:  [[8.88360739e-01 6.10858202e-04 5.51566482e-03 ... 6.85341365e-05
  1.33574009e-04 1.76202652e-09]
 [9.83343959e-01 5.05238771e-04 6.87873363e-03 ... 8.11308622e-04
  1.69098377e-03 6.68148870e-09]
 [9.99457598e-01 4.88628416e-10 2.02327305e-06 ... 7.14638140e-15
  2.22996088e-09 3.78388227e-28]
 ...
 [9.99615252e-01 6.78967496e-08 2.38388777e-04 ... 1.05758804e-10
  9.88921784e-07 3.07572227e-21]
 [9.70586896e-01 4.36156988e-04 6.13066554e-03 ... 9.08980655e-05
  4.99874353e-04 2.73381456e-10]
 [6.01508558e-01 3.09070945e-03 1.04251504e-03 ... 2.44259834e-03
  6.58601522e-04 2.14260069e-07]]
[del] Will also save the embedding, to:  ../data/sp/yamnet/query_embedding/description/fast/12.h5
[del] will it be true:  True
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/13.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/description/fast/13.h5 False
[del] if statement TRUE! reading  ../data/sp/podcasts-no-audio-13GB/topic_t

...

[del] scores:  [[9.9110240e-01 9.8019838e-04 4.7442615e-03 ... 1.9506724e-05
  1.9273162e-04 1.7349959e-10]
 [9.9993736e-01 1.2616393e-12 3.9459965e-06 ... 7.3476063e-20
  5.8841495e-12 0.0000000e+00]
 [9.9627769e-01 4.6893169e-06 1.4088750e-03 ... 9.7779298e-07
  6.1816427e-05 1.2373887e-11]
 ...
 [9.9901903e-01 1.1272497e-05 1.2898445e-02 ... 8.0191169e-09
  6.5286581e-06 6.3305302e-20]
 [9.9962091e-01 1.1877092e-06 1.1421740e-03 ... 4.7198370e-07
  3.5535853e-05 2.0571591e-13]
 [9.5284247e-01 2.0133853e-03 5.9469640e-03 ... 6.8283081e-04
  1.0972321e-03 5.9034406e-08]]
[del] Will also save the embedding, to:  ../data/sp/yamnet/query_embedding/description/fast/22.h5
[del] will it be true:  True
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/23.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/description/fast/23.h5 False
[del] if statement TRUE! reading  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/23.

 46%|████▌     | 57/124 [00:15<00:22,  2.96it/s]

[del] scores:  [[9.9996161e-01 1.0538135e-12 2.1414344e-05 ... 1.8091232e-22
  1.6553874e-13 0.0000000e+00]
 [9.9907827e-01 2.9383166e-06 2.2701919e-03 ... 6.0562195e-07
  5.1643648e-05 6.8722046e-14]
 [9.9948120e-01 1.2730441e-08 8.5023046e-04 ... 5.5156167e-13
  3.6901469e-07 3.6164817e-26]
 ...
 [9.9961758e-01 3.9452641e-07 3.5425127e-03 ... 5.3997002e-07
  7.1598020e-05 8.6698134e-15]
 [9.4612205e-01 3.8158894e-04 3.6031008e-03 ... 1.8116534e-03
  1.7130375e-03 6.7189461e-07]
 [2.1213445e-08 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
[del] Will also save the embedding, to:  ../data/sp/yamnet/query_embedding/description/fast/23.h5
[del] will it be true:  True
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/37.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/description/fast/37.h5 False
[del] if statement TRUE! reading  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/fast/37.

100%|██████████| 124/124 [00:15<00:00,  7.88it/s]

[del] scores:  [[9.93117571e-01 1.26332045e-04 3.20485234e-03 ... 1.68428269e-05
  1.15390314e-04 9.48636170e-11]
 [9.42268014e-01 4.80675697e-03 4.88695502e-03 ... 1.14744902e-03
  1.44413114e-03 1.12172586e-07]
 [9.97702003e-01 7.73039233e-07 5.84763288e-03 ... 1.95568660e-07
  2.87592411e-04 1.27046658e-14]
 ...
 [9.99837399e-01 4.55559723e-09 1.62661076e-04 ... 4.19442172e-12
  3.33994990e-07 1.26143304e-26]
 [9.96400714e-01 7.42881675e-05 1.11560822e-02 ... 1.15818410e-04
  2.66984105e-03 5.46853604e-11]
 [9.99554038e-01 6.47615934e-06 5.41880727e-03 ... 4.76181611e-07
  3.22166197e-05 4.33278020e-15]]
[del] Will also save the embedding, to:  ../data/sp/yamnet/query_embedding/description/fast/37.h5
[del] will it be true:  False
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/query/slow/8.ogg True
Should not exist:  ../data/sp/yamnet/query_embedding/query/slow/8.h5 True
[del] will it be true:  False
Should exist:  ../data/sp/podcasts-no-audio-13GB/topic_task

In [30]:

# test_input_path = 'test/query/fast/8.ogg'
# test2 = os.path.abspath(test_input_path)
# print(test2)

# # test2 = '../8.ogg'

# # test2 = '../data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/slow/8.ogg'
# wav_data, sr = sf.read(test2, dtype=np.int16)



# /Users/casper/Documents/UvA\ master/b23456_thesis/msc_thesis/code/data/sp/podcasts-no-audio-13GB/topic_task/query_audio/description/slow/8.ogg




/Users/casper/Documents/UvAmaster/b23456_thesis/msc_thesis/code/scripts/test/query/fast/8.ogg
